In [1]:
# API：
# tf.random.categorical #随机采样
# tf.expand_dims(input_eval, 0)#增加维度
# tf.squeeze(predictions, 0)#减少维度

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import tensorflow as tf
from tensorflow import keras
import os
print(tf.__name__, tf.__version__)

tensorflow 2.1.0


In [3]:
# import requests
# import os
# url = 'http://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt'

# r = requests.get(url)
# # print(r.text)
# with open('data/shakespeare.txt','w') as f:
#     f.write(r.text)

In [2]:
# http://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt
input_filepath = "data/shakespeare.txt"
text = open(input_filepath, 'r').read()

print(len(text))
print(text[0:100])

1115394
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [3]:
# 数据处理
# 1.generate vocab
# 2.build mapping char ->id
# 3.data -> id_data
# 4.abcd  -> bcd<eos>
vocab = sorted(set(text))

print(len(vocab))
print(vocab)


65
['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [4]:
char_to_idx = {char: idx for idx, char in enumerate(vocab)}
#enumerate() 函数用于将一个可遍历的数据对象(如列表、元组或字符串)组合为一个索引序列
print(char_to_idx)


{'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'A': 13, 'B': 14, 'C': 15, 'D': 16, 'E': 17, 'F': 18, 'G': 19, 'H': 20, 'I': 21, 'J': 22, 'K': 23, 'L': 24, 'M': 25, 'N': 26, 'O': 27, 'P': 28, 'Q': 29, 'R': 30, 'S': 31, 'T': 32, 'U': 33, 'V': 34, 'W': 35, 'X': 36, 'Y': 37, 'Z': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, 'h': 46, 'i': 47, 'j': 48, 'k': 49, 'l': 50, 'm': 51, 'n': 52, 'o': 53, 'p': 54, 'q': 55, 'r': 56, 's': 57, 't': 58, 'u': 59, 'v': 60, 'w': 61, 'x': 62, 'y': 63, 'z': 64}


In [5]:
idx_to_char = np.array(vocab)
print(idx_to_char)

['\n' ' ' '!' '$' '&' "'" ',' '-' '.' '3' ':' ';' '?' 'A' 'B' 'C' 'D' 'E'
 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R' 'S' 'T' 'U' 'V' 'W'
 'X' 'Y' 'Z' 'a' 'b' 'c' 'd' 'e' 'f' 'g' 'h' 'i' 'j' 'k' 'l' 'm' 'n' 'o'
 'p' 'q' 'r' 's' 't' 'u' 'v' 'w' 'x' 'y' 'z']


In [6]:
text_as_int = np.array([char_to_idx[c] for c in text])
print(text_as_int[0:100])

[18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43
  1 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43
 39 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49
  6  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10
  0 37 53 59]


In [7]:
def split_input_target(id_text):
    # 4.abcde  -> abcd, bcde
    return id_text[0:-1],  id_text[1:]
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
seq_length = 100
seq_dataset = char_dataset.batch(seq_length + 1, 
                                                 drop_remainder = True)
# drop_remainder 做batch最后如果不够就丢掉
for ch_id in char_dataset.take(2):
    print(ch_id, idx_to_char[ch_id.numpy()])
for seq_id in seq_dataset.take(2):
    print(seq_id)
    print(repr(''.join(idx_to_char[seq_id.numpy()])))# repr 显示特殊字符

tf.Tensor(18, shape=(), dtype=int64) F
tf.Tensor(47, shape=(), dtype=int64) i
tf.Tensor(
[18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43
  1 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43
 39 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49
  6  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10
  0 37 53 59  1], shape=(101,), dtype=int64)
'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
tf.Tensor(
[39 56 43  1 39 50 50  1 56 43 57 53 50 60 43 42  1 56 39 58 46 43 56  1
 58 53  1 42 47 43  1 58 46 39 52  1 58 53  1 44 39 51 47 57 46 12  0  0
 13 50 50 10  0 30 43 57 53 50 60 43 42  8  1 56 43 57 53 50 60 43 42  8
  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 18 47 56 57 58  6  1
 63 53 59  1 49], shape=(101,), dtype=int64)
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'


In [8]:
seq_dataset = seq_dataset.map(split_input_target)

for item_input, item_output in seq_dataset.take(2):
    print(item_input.numpy())
    print(item_output.numpy())

[18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43
  1 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43
 39 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49
  6  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10
  0 37 53 59]
[47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43  1
 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43 39
 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49  6
  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10  0
 37 53 59  1]
[39 56 43  1 39 50 50  1 56 43 57 53 50 60 43 42  1 56 39 58 46 43 56  1
 58 53  1 42 47 43  1 58 46 39 52  1 58 53  1 44 39 51 47 57 46 12  0  0
 13 50 50 10  0 30 43 57 53 50 60 43 42  8  1 56 43 57 53 50 60 43 42  8
  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 18 47 56 57 58  6  1
 63 53 59  1]
[56 43  1 39 50 50  1 56 43 57 53 50 60 43 42  1 56 39 58 46 43 56  1 58
 53  1 42

In [9]:
batch_size = 64
buffer_size = 10000
seq_dataset = seq_dataset.shuffle(buffer_size).batch(
                      batch_size, drop_remainder = True)

In [12]:
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

def build_model(vocab_size, embedding_dim, rnn_units,batch_size):
    model = keras.models.Sequential([
        keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape = [batch_size, None]),
        keras.layers.Bidirectional(keras.layers.LSTM(units=32,return_sequences=False)),

        keras.layers.SimpleRNN(units= rnn_units,
                                          return_sequences = True),
        keras.layers.Dense(vocab_size)
    ])
    return model

model = build_model(
        vocab_size = vocab_size,
        embedding_dim=embedding_dim,
        rnn_units = rnn_units,
        batch_size=batch_size
)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
simple_rnn (SimpleRNN)       (64, None, 1024)          1311744   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 1,395,009
Trainable params: 1,395,009
Non-trainable params: 0
_________________________________________________________________


In [13]:
# 用定义好的模型做预测
for input_example_batch, target_example_batch in seq_dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape)

(64, 100, 65)


In [14]:
#随机采样，random sampling
#greedy贪心策略，random随机策略
sample_indices = tf.random.categorical(
    logits = example_batch_predictions[0], num_samples = 1)
print(sample_indices)
#(100, 65) -> (100, 1)
sample_indices = tf.squeeze(sample_indices, axis =1)

tf.Tensor(
[[63]
 [59]
 [ 3]
 [55]
 [62]
 [60]
 [53]
 [ 8]
 [13]
 [ 7]
 [56]
 [37]
 [11]
 [36]
 [35]
 [52]
 [50]
 [38]
 [58]
 [16]
 [21]
 [ 3]
 [44]
 [50]
 [50]
 [36]
 [27]
 [34]
 [33]
 [ 8]
 [20]
 [42]
 [11]
 [59]
 [46]
 [ 2]
 [28]
 [ 8]
 [52]
 [54]
 [56]
 [28]
 [ 9]
 [59]
 [49]
 [ 8]
 [36]
 [ 4]
 [44]
 [34]
 [62]
 [ 0]
 [22]
 [40]
 [19]
 [57]
 [ 3]
 [27]
 [47]
 [35]
 [ 3]
 [16]
 [44]
 [62]
 [25]
 [48]
 [44]
 [25]
 [41]
 [22]
 [33]
 [62]
 [26]
 [38]
 [18]
 [24]
 [12]
 [ 7]
 [14]
 [18]
 [58]
 [10]
 [ 3]
 [64]
 [54]
 [54]
 [ 0]
 [27]
 [52]
 [47]
 [41]
 [ 5]
 [32]
 [64]
 [26]
 [56]
 [63]
 [50]
 [31]
 [44]], shape=(100, 1), dtype=int64)


In [15]:
print('Input', repr("".join(idx_to_char[input_example_batch[0]])))
print()
print('Output', repr("".join(idx_to_char[target_example_batch[0]])))
print()
print('Predictions', repr("".join(idx_to_char[sample_indices])))
# 模型还没有训练，Predictions打出无需字母

Input 'ORK:\nWhat, will you go unto the Tower, my lord?\n\nPRINCE EDWARD:\nMy lord protector needs will have it'

Output 'RK:\nWhat, will you go unto the Tower, my lord?\n\nPRINCE EDWARD:\nMy lord protector needs will have it '

Predictions "yu$qxvo.A-rY;XWnlZtDI$fllXOVU.Hd;uh!P.nprP3uk.X&fVx\nJbGs$OiW$DfxMjfMcJUxNZFL?-BFt:$zpp\nOnic'TzNrylSf"


In [16]:
# 自定义损失函数
def loss(labels, logits):
    return keras.losses.sparse_categorical_crossentropy(
            labels, logits, from_logits = True)

model.compile(optimizer = 'adam', loss = loss)
example_loss = loss(target_example_batch, example_batch_predictions)
print(example_loss.shape)
print(example_loss.numpy().mean())


(64, 100)
4.190553


In [17]:
# 保存模型
output_dir = 'data/text_generation_checkpoints'
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
    
checkpoint_prefix = os.path.join(output_dir, 'ckpt_{epoch}')
checkpoint_callback = keras.callbacks.ModelCheckpoint(
        filepath = checkpoint_prefix,
        save_weights_only = True)

epochs = 30
history = model.fit(seq_dataset, epochs = epochs,
                           callbacks = [checkpoint_callback])

Train for 172 steps
Epoch 1/30
172/172 [==============================] - 9s 54ms/step - loss: 2.9785
Epoch 2/30
172/172 [==============================] - 9s 50ms/step - loss: 2.2581
Epoch 3/30
172/172 [==============================] - 9s 54ms/step - loss: 2.0484
Epoch 4/30
172/172 [==============================] - 9s 53ms/step - loss: 1.8994
Epoch 5/30
172/172 [==============================] - 9s 51ms/step - loss: 1.7845
Epoch 6/30
172/172 [==============================] - 9s 53ms/step - loss: 1.6953
Epoch 7/30
172/172 [==============================] - 9s 53ms/step - loss: 1.6251
Epoch 8/30
172/172 [==============================] - 9s 53ms/step - loss: 1.5690
Epoch 9/30
172/172 [==============================] - 9s 51ms/step - loss: 1.5256
Epoch 10/30
172/172 [==============================] - 9s 51ms/step - loss: 1.4881
Epoch 11/30
172/172 [==============================] - 9s 50ms/step - loss: 1.4586
Epoch 12/30
172/172 [==============================] - 9s 51ms/step - loss: 

In [18]:
#查看最新保存的模型tf.train.latest_checkpoint
tf.train.latest_checkpoint(output_dir)

'data/text_generation_checkpoints/ckpt_30'

In [19]:
# 从checkpoint载入模型
model2  = build_model(vocab_size,
                                 embedding_dim,
                                 rnn_units,
                                 batch_size = 1)
model2.load_weights(tf.train.latest_checkpoint(output_dir))
model2.build(tf.TensorShape([1, None]))
# 文本生成流程：
# start ch sequence A,
# A -> model -> b
# A -> append(b) -> B
# B -> model -> c
# B -> append(c) -> C
# C -> model -> ....

model2.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (1, None, 1024)           1311744   
_________________________________________________________________
dense_1 (Dense)              (1, None, 65)             66625     
Total params: 1,395,009
Trainable params: 1,395,009
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# 利用模型生成文本
def generata_text(model, start_string, num_genetate = 1000):
    input_eval = [char_to_idx[ch] for ch in start_string]
    input_eval = tf.expand_dims(input_eval, 0)#增加维度
    
    text_generated = []
    model.reset_states()
    
    for _ in range(num_genetate):
        # 1. model inference -> predictions
        # 2. sample -> ch -> text_generated.
        # 3. update input_evel
        # predictions : [batch_size, input_eval_len, vocab_size]
        predictions = model(input_eval)
        # predictions : [input_eval_len, vocab_size]
        predictions = tf.squeeze(predictions, 0)#减少维度
        # predictions : [input_eval_len, 1]
        # a b c ->  b c d
        predictions_id = tf.random.categorical(
                predictions, num_samples=1)[-1,0].numpy()
        text_generated.append(idx_to_char[predictions_id])
        input_evel = tf.expand_dims([predictions_id], 0)
    return start_string + ''.join(text_generated)

new_text = generata_text(model2, "All: ")
print(new_text)